# Calculate NPV

## Setup

In [92]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [94]:
# Import libraries
import pandas as pd
import numpy as np
from investing.config.file_paths import input_data, output_data
from investing.tools.functions import npv_calc, pv_full_output

<IPython.core.display.Javascript object>

## Configure

In [95]:
# File names
parameters_file = "params_regular.csv"
fcf_start_values = 'fcf_start_data.csv'
npv_file = "npv.csv"

<IPython.core.display.Javascript object>

## Calculate NPV

In [96]:
# Load dataframe of all input values
df_params = pd.read_csv(input_data/parameters_file)
df_fcf_start = pd.read_csv(output_data/ fcf_start_values)

<IPython.core.display.Javascript object>

In [97]:
# Build the dataframe
df = pd.merge(df_params, df_fcf_start,'outer','ticker')

<IPython.core.display.Javascript object>

In [98]:
# Set the Discount Rate
df['discount_rate'] = 10

<IPython.core.display.Javascript object>

In [99]:
# Calculate the NPV
for pvalue in ['p10','p50','p90']:
    starting_value = df['start_value']
    discount_rate = df['discount_rate']
    perpetual_rate = df[f'{pvalue}_terminal']
    growth_stage1 = df[f'{pvalue}_stage1']
    growth_stage2 = df[f'{pvalue}_stage2']
    
    df[f'npv_{pvalue}'] = np.vectorize(npv_calc)(starting_value, discount_rate, growth_stage1, growth_stage2, perpetual_rate)

<IPython.core.display.Javascript object>

In [100]:
# Calculate the present value forecasts
pv_forecasts = {}
temp_dict = {}
for index, row in df.iterrows():
    ticker = row['ticker']
    for pvalue in ['p10','p50','p90']:
        starting_value = row['start_value']
        discount_rate = row['discount_rate']
        perpetual_rate = row[f'{pvalue}_terminal']
        growth_stage1 = row[f'{pvalue}_stage1']
        growth_stage2 = row[f'{pvalue}_stage2']

        # Set the PV forecast to a temporary dictionary
        temp = pv_full_output(starting_value, discount_rate, growth_stage1, growth_stage2, perpetual_rate)
        # Extract the PV forecast dataframe from the dictionary and set to the temparory dict
        temp_dict[pvalue] = temp['full_pv']
    # Set temp dict to the ticker
    pv_forecasts[ticker] = temp_dict

<IPython.core.display.Javascript object>

In [101]:
pv_forecasts['ACN']['p50']

,years,value,PV
0,1.0,7.365600,6.696000
1,2.0,7.954848,6.574255
2,3.0,8.591236,6.454723
3,4.0,9.278535,6.337364
4,5.0,10.020817,6.222139
5,6.0,10.622067,5.995880
6,7.0,11.259391,5.777848
7,8.0,11.934954,5.567744
8,9.0,12.651051,5.365281
9,10.0,13.410114,5.170180


<IPython.core.display.Javascript object>

In [102]:
# Calculate Risked NPV
df['risked_npv'] = 0.3 * df['npv_p10'] + 0.4 * df['npv_p50'] + 0.3 * df['npv_p90']

<IPython.core.display.Javascript object>

## Export Data

In [103]:
df.to_csv(output_data/npv_file, index=False)

<IPython.core.display.Javascript object>